In [2]:
%%capture
!pip install unsloth

!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [4]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import random

from datasets import load_dataset
open_instruct_dataset = load_dataset("5CD-AI/Vietnamese-Ecommerce-Alpaca", split="train")

In [5]:
open_instruct_dataset[1]

{'output': 'Sử dụng túi giặt có thể mang lại một số lợi ích, bao gồm:\n\n- Bảo vệ quần áo: Túi giặt có thể giúp bảo vệ quần áo khỏi bị hư hỏng trong quá trình giặt, đặc biệt là đối với quần áo mỏng manh hoặc dễ bị rách.\n\n- Giúp giặt sạch hơn: Túi giặt có thể giúp quần áo được giặt sạch hơn bằng cách tạo ra chuyển động tốt hơn trong máy giặt và giúp chất tẩy rửa tiếp xúc với quần áo tốt hơn.\n\n- Giảm xoắn rối: Túi giặt có thể giúp giảm xoắn rối quần áo trong quá trình giặt, giúp quần áo dễ ủi hơn và trông đẹp hơn.\n\n- Tiết kiệm thời gian: Túi giặt có thể giúp tiết kiệm thời gian bằng cách giảm thời gian ủi quần áo và giúp quần áo khô nhanh hơn.\n\n- Giảm tiếng ồn: Túi giặt có thể giúp giảm tiếng ồn từ máy giặt, đặc biệt là đối với máy giặt cũ hoặc bị hỏng.',
 'instruction': 'Nói cho tôi biết tại sao nên sử dụng túi giặt.',
 'input': ''}

In [6]:
max_seq_length = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
EOS_TOKEN = tokenizer.eos_token

In [8]:
def format_row_as_instruction_prompt(example):
    has_input = example.get('input', None) is not None

    if has_input:
        primer_prompt = ("Below is an instruction that describes a task, paired with an input "
                         "that provides further context. Write a response that appropriately completes the request.")
        input_template = f"### Input: \n{example['input']}\n\n"
    else:
        primer_prompt = ("Below is an instruction that describes a task. "
                         "Write a response that appropriately completes the request.")
        input_template = ""

    instruction_template = f"### Instruction: \n{example['instruction']}\n\n"

    if example.get('output', None):
        response_template = f"### Response: \n{example['output']}\n\n"
    else:
        response_template = ""

    # Wrap the resulting string in a list
    return f"{primer_prompt}\n\n{instruction_template}{input_template}{response_template}"


def formatting_prompt(examples):
    """
    Formats a batch of examples using the format_row_as_instruction_prompt logic.

    Args:
        examples (dict): A dictionary with keys "instruction", "input", and "output".

    Returns:
        dict: A dictionary containing a list of formatted text prompts.
    """
    formatted_texts = []

    for instruction, input_, output in zip(
        examples.get("instruction", []),
        examples.get("input", []),
        examples.get("output", []),
    ):
        # Create a single example dictionary
        example = {
            "instruction": instruction,
            "input": input_,
            "output": output
        }

        # Use format_row_as_instruction_prompt to format the example
        formatted_text = format_row_as_instruction_prompt(example) + EOS_TOKEN
        formatted_texts.append(formatted_text)

    return {"text": formatted_texts}

In [9]:
training_data = open_instruct_dataset.map(formatting_prompt, batched=True)

In [10]:
training_data[1]["text"]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: \nNói cho tôi biết tại sao nên sử dụng túi giặt.\n\n### Input: \n\n\n### Response: \nSử dụng túi giặt có thể mang lại một số lợi ích, bao gồm:\n\n- Bảo vệ quần áo: Túi giặt có thể giúp bảo vệ quần áo khỏi bị hư hỏng trong quá trình giặt, đặc biệt là đối với quần áo mỏng manh hoặc dễ bị rách.\n\n- Giúp giặt sạch hơn: Túi giặt có thể giúp quần áo được giặt sạch hơn bằng cách tạo ra chuyển động tốt hơn trong máy giặt và giúp chất tẩy rửa tiếp xúc với quần áo tốt hơn.\n\n- Giảm xoắn rối: Túi giặt có thể giúp giảm xoắn rối quần áo trong quá trình giặt, giúp quần áo dễ ủi hơn và trông đẹp hơn.\n\n- Tiết kiệm thời gian: Túi giặt có thể giúp tiết kiệm thời gian bằng cách giảm thời gian ủi quần áo và giúp quần áo khô nhanh hơn.\n\n- Giảm tiếng ồn: Túi giặt có thể giúp giảm tiếng ồn từ máy giặt, đặc biệt là đối với máy 

In [11]:
training_data

Dataset({
    features: ['output', 'instruction', 'input', 'text'],
    num_rows: 69303
})

In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)


Unsloth 2024.12.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
train_subset = training_data.select(range(1000))

In [15]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_subset,  # train set
    dataset_text_field="text", # field to use
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=1e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4, # giả lập batch_size là 16
        num_train_epochs=1,
        fp16=True,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        warmup_steps=10,
        output_dir="output",
        seed=1512
    ),
)




Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
train_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 281 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 9
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.370300
2,1.364000
3,1.381400
4,1.282500
5,1.282200
6,1.241300
7,1.185000
8,1.114600
9,1.078200


### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

In [21]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [35]:
!nvidia-smi

Thu Jan  2 07:38:09 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0              32W /  70W |  12603MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# restart session and run predict
cause out of ram, I tried some methods to prevent it but it's not work!

Remember to save your content/lora_model before restart runtime, or you will loss all your process!

In [6]:

from transformers import AutoTokenizer, BitsAndBytesConfig

# Define quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    llm_int8_enable_fp32_cpu_offload=True,  # Allow CPU offloading for FP32 modules
)

# Load the LoRA model and apply quantization
if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="/content/lora_model",  # Path to LoRA model
        max_seq_length=128,
        quantization_config=quantization_config,  # Apply quantization config
        device_map="cuda",  # Automatically distribute model across CPU and GPU
    )
    FastLanguageModel.for_inference(model)  # Enable faster inference

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Tại sao tôi nên dùng máy giặt?",  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)


==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tại sao tôi nên dùng máy giặt?

### Input:


### Response:
Máy giặt giúp bạn tiết kiệm thời gian và công sức. Bạn có thể giặt quần áo một lần và không cần phải giặt lại nhiều lần. Máy giặt cũng giúp quần áo sạch hơn và mềm hơn. Ngoài ra, máy giặt cũng giúp bạn tiết kiệm tiền. Bạn có thể giặt quần áo nhiều lần và không cần phải mua quần áo mới.

<|en